## YouTube Scraping with Beautiful Soup and Selenium

In [1]:
banner = """

   dM  dMP    .aMMMb    dMP dMP  dMMMMMMP    dMP dMP     dMMMMb     dMMMMMP        .dMMMb    .aMMMb     dMMMMb     .aMMMb     dMMMMb     dMMMMMP     dMMMMb 
  dMP.dMP    dMP dMP   dMP dMP     dMP      dMP dMP     dMP .MP    dMP            dMP   V   dMP  MP    dMP dMP    dMP dMP    dMP dMP    dMP         dMP dMP 
  VMMMMP    dMP dMP   dMP dMP     dMP      dMP dMP     dMMMMK"    dMMMP           VMMMb    dMP        dMMMMK"    dMMMMMP    dMMMMP"    dMMMP       dMMMMK"  
n  .dMP    dMP aMP   dMP aMP     dMP      dMP aMP     dMP .MF    dMP            n   dMP   dMP  MP    dMP AMF    dMP dMP    dMP        dMP         dMP AMF   
VMMMP"     VMMMP"    VMMMP"     dMP       VMMMP"     dMMMMP"    dMMMMMP         VMMMP"    VMMMP"    dMP  MP    dMP dMP    dMP        dMMMMMP     dMP  MP    

"""

### 패키지 및 사전 세팅

In [2]:
import time
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from time import sleep

import warnings 
warnings.filterwarnings('ignore')

class bcolors: # 4-bit Colours ANSI
    OKMAGENTA = '\033[95m'
    OKBLUE = '\033[94m[SCRAPING]'
    OKGREEN = '\033[92m'
    OKYELLOW = '\033[93m'
    OKCYAN = '\033[96m'
    OKRED = '\033[91m'
    ENDC = '\033[0m'

bc = bcolors

In [3]:
df = pd.DataFrame(columns=('title', 'url'))

### 변수 값 작성 (채널과 필요한 데이터 양 변경 가능)

In [4]:
channel_name = 'sbs8news' # 채널 명
end_point = 1000 # 필요한 데이터 수 <-- 스크랩하고 싶은만큼 바꿔주세요

### 셀레니움으로 유튜브 속성값 가져오기

In [5]:
options = webdriver.ChromeOptions()
options.add_argument('headless') # 크롬드라이버 창을 열지 않고 백그라운드에서 실행하는 옵션입니다.
options.add_experimental_option('excludeSwitches', ['enable-logging']) # 불필요한 로그는 숨기는 옵션입니다.
driver = webdriver.Chrome(options=options)

channel_url = 'https://www.youtube.com/{}/videos'.format(channel_name)
driver.get(channel_url)

### 뷰티풀수프를 이용한 데이터 크롤링

In [6]:
print(bc.OKMAGENTA + banner + bc.ENDC) # 채원님이 원하셨던 핑크!

base_url = 'https://www.youtube.com'
prev_scroll = driver.execute_script('return document.documentElement.scrollHeight') # 이전 스크롤 위치 입니다.
prev_elements = []

cnt = 0
while True:
    try:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);") # 스크롤을 제어하여 최대한 내립니다.
        time.sleep(2) # 데이터가 로딩되기까지 sleep

        url_source = driver.page_source
        soup = BeautifulSoup(markup=url_source, features='html.parser')

        curr_elements = soup.find_all('a', id='video-title')

        for i in range(len(prev_elements), len(curr_elements)):
            if cnt == end_point: # 종료지점에서 드라이브를 close를 하고 오류를 발생시킵니다. <-- 종료 포인트를 주고 싶지 않으면 이 if문을 삭제하시면 됩니다.
                driver.close()
                break
            else:
                cnt += 1
                elements = curr_elements[i]
                title = elements.text # 필요없으면 제거해도 됩니다. 다만, 아래 데이터 프레임에 append하는 부분 수정해주세요.
                url = base_url + str(elements).split(' href=')[1].split(' ')[0][1:-1]
                df = df.append({'title': title, 'url': url}, ignore_index=True) # 제목 제거 시에 수정 필요합니다.
                
                message = ' {}% | Scraping the "{}" Channel...'.format(int(cnt/end_point*100), channel_name.upper())
                print('\r' + bc.OKBLUE + message + bc.ENDC, end="")
        
        curr_scroll = driver.execute_script('return document.documentElement.scrollHeight') # 현 시점에서 스크롤을 최대로 내린 스코롤 위치 입니다.
        if curr_scroll == prev_scroll: # 현재 스크롤 위치가 이전 스크롤 위치와 같으면 드라이브를 close를 하고 오류를 발생시킵니다.
            driver.close()
            break
            
        prev_scroll = curr_scroll
        prev_elements = curr_elements
            
    except:
        print("\n" + bc.OKGREEN + '----------Done!----------'+ bc.ENDC)
        break




   dM  dMP    .aMMMb    dMP dMP  dMMMMMMP    dMP dMP     dMMMMb     dMMMMMP        .dMMMb    .aMMMb     dMMMMb     .aMMMb     dMMMMb     dMMMMMP     dMMMMb 
  dMP.dMP    dMP dMP   dMP dMP     dMP      dMP dMP     dMP .MP    dMP            dMP   V   dMP  MP    dMP dMP    dMP dMP    dMP dMP    dMP         dMP dMP 
  VMMMMP    dMP dMP   dMP dMP     dMP      dMP dMP     dMMMMK"    dMMMP           VMMMb    dMP        dMMMMK"    dMMMMMP    dMMMMP"    dMMMP       dMMMMK"  
n  .dMP    dMP aMP   dMP aMP     dMP      dMP aMP     dMP .MF    dMP            n   dMP   dMP  MP    dMP AMF    dMP dMP    dMP        dMP         dMP AMF   
VMMMP"     VMMMP"    VMMMP"     dMP       VMMMP"     dMMMMP"    dMMMMMP         VMMMP"    VMMMP"    dMP  MP    dMP dMP    dMP        dMMMMMP     dMP  MP    


[SCRAPING] 100% | Scraping the "SBS8NEWS" Channel...
----------Done!----------


### 결과 출력

In [7]:
df

,title,url
0,[날씨] 서울 낮 최고 34도…곳곳 요란한 소나기 / SBS,https://www.youtube.com/watch?v=LsQtYR6XqGY
1,"'국가부도' 스리랑카 대통령, 반정부 시위에 사임 선언 / SBS",https://www.youtube.com/watch?v=_oZ6TU8QVBI
2,강남 영화관 화재에 80여 명 대피…인명 피해 없어 / SBS,https://www.youtube.com/watch?v=HSKEhVSNUKk
3,한 달 반 만에 다시 2만 명대…13일 대응 방안 발표 / SBS,https://www.youtube.com/watch?v=DOC-NZIiblg
4,"""어머니가 빠진 종교 단체에 원한""…아베, 12일 가족장 / SBS",https://www.youtube.com/watch?v=GNm0iqgDnAE
...,...,...
995,더불어민주당 비상대책위원회 / SBS,https://www.youtube.com/watch?v=qL7xvJRM38o
996,"""'타닥타닥' 빗소리 들으니 이 짝꿍 생각나요"" / SBS / 친절한 경제",https://www.youtube.com/watch?v=zpjImbLsX5k
997,"윤 대통령, 스페인 방문 2일차- 국왕 만찬 참석 등 외교 일정 / SBS",https://www.youtube.com/watch?v=gnJ1CgxRpKU
998,"""말하지 않아도 신난다""…언어의 경계 허문 '파란 남자들' / SBS / 초대석",https://www.youtube.com/watch?v=bzplJrQSzv4


In [8]:
len(list(df['title'].unique()))

996

In [9]:
df[df['title'].duplicated() == True] # 제목에서 중복 값이 존재하는데 이는 사이트에서도 중복으로 나타남으로 문제 없습니다.

,title,url
175,"'화제의 장타자' 윤이나…""비결은 '지면 반력'"" / SBS",https://www.youtube.com/watch?v=kL6VnGlqpuI
627,"박태환 ""황선우는 저보다 뛰어난 선수…응원할게요"" / SBS",https://www.youtube.com/watch?v=bA2T9nrwLwA
633,'이자 장사' 비판에 대출금리↓ · 예금금리↑ / SBS,https://www.youtube.com/watch?v=sjtywcnw2NI
772,올해 상반기 무역적자 103억 달러…역대 최대 규모 / SBS,https://www.youtube.com/watch?v=cH_T5McxQNI


In [10]:
len(list(df['url'].unique()))

1000

In [11]:
# df.to_csv('./youtubeScrap.csv',index = False, encoding='utf-8-sig')